# Content-Based Filtering

Filtrování možných doporučování na základě podobnosti obsahu je jedním ze základních principů doporučovacích systémů. Celou strategií je najít takové dvojice položek, které jsou si nejvíce podobné na základě k nim přiřazených atributů.

Jako příklad bychom si mohli uvést následující tvrzení:

`Chléb Benešov` s atributy `(700 g, potraviny, pečivo, chléb, moučné produkty)` bude jistě poměrně podobný položce `Chléb Šumava` s atributy `(1600 g, potraviny, pečivo, chléb, moučné produkty)`. Jistou podobnost bychom mohli najít i s `Bageta celozrnná` s atributy `(240 g, potraviny, pečivo, bageta, moučné produkty)`. 

Možná by nám dávalo smysl doporučovat i `Mléko` definované jako `(1 l, potraviny, mléčné výrobky)`. Nicméně bychom mezi těmito produkty neměli hledat podobnost s `Čistič myčky`, u kterého bychom měli přiřazeny příznaky `(150 ml, potřeby pro domácnost, kuchyně, čistící prostředky, potřeby na mytí nádobí)`.

Otázkou pak je, jak hledat takto podobné dvojice položek, které většinou bývají reprezentovány převážně kategoriálními atributy. My se v této ukázce zaměříme na doporučování námi vybudované databáze hodnocení filmů na základě podobnosti mezi dvojicemi filmů přes jejich atributy.

## Zadání problému

V této ukázce bychom chtěli doporučovat filmy jen na základě žánrů k těmto přiřazených. Cílem tedy je nalézt filmy, které jsou si co do žánrového (kategoriálního) přiřazení nejpodobnější.

## Načtení dat

První fází je načtení zdrojových dat, která budeme chtít zpracovávat. Vstupní data jsou ve formátu `json` coby seznam položek o následujících atributech:
- `_id` typu `str` (unikátní identifikátor záznamu filmu)
- `name` typu `str` (název filmu)
- `year` typu `int` (rok natočení filmu)
- `genres` typu `str` (seznam žánrů filmu oddělený znakem lomítka `/`)
- `csfdUrl` typu `str` (reference na ČSFD) 

In [1]:
import json
import random
from dataclasses import dataclass

In [2]:
def load_file(file_path: str):
    with open(file_path) as file:
        data = json.load(file)
        file.close()
        return data

movies_data = load_file("./data/movies.json")

In [3]:
movies_data[:3]

[{'_id': '64d2a7963b7afeb6a7ff6b70',
  'name': 'Vykoupení z věznice Shawshank',
  'year': 1994,
  'genres': 'Drama / Krimi / Vězení',
  'csfdUrl': '2294-vykoupeni-z-veznice-shawshank'},
 {'_id': '64d2a7f53b7afeb6a7ff6b77',
  'name': 'Nedotknutelní',
  'year': 2011,
  'genres': 'Komedie / Drama / Životopisný',
  'csfdUrl': '306731-nedotknutelni'},
 {'_id': '64d2b5d6aacb468a3ff1a9a8',
  'name': 'Kmotr',
  'year': 1972,
  'genres': 'Drama / Krimi',
  'csfdUrl': '1644-kmotr'}]

Pro prvotní manipulaci s daty využijme definice třídy, která nám usnadní a zpřehlední naše úvodní transformace. Využijme k tomu třídního dekorátoru `dataclass` z balíčku `dataclasses`.

In [4]:
@dataclass
class Movie:
    movie_id: str
    name: str
    year: int
    genres: list[str]

    def is_of_genre(self, genre: str) -> bool:
        return genre in self.genres

In [5]:
movies = []

def process_genres(genres_str: str) -> list[str]:
    return [genre.strip() for genre in genres_str.split("/")]

for movie in movies_data:
    movies.append(
        Movie(
            movie_id=movie["_id"],
            name=movie["name"],
            year=movie["year"],
            genres=process_genres(movie["genres"])
        )
    )

for movie in movies[:3]:
    print(movie)

Movie(movie_id='64d2a7963b7afeb6a7ff6b70', name='Vykoupení z věznice Shawshank', year=1994, genres=['Drama', 'Krimi', 'Vězení'])
Movie(movie_id='64d2a7f53b7afeb6a7ff6b77', name='Nedotknutelní', year=2011, genres=['Komedie', 'Drama', 'Životopisný'])
Movie(movie_id='64d2b5d6aacb468a3ff1a9a8', name='Kmotr', year=1972, genres=['Drama', 'Krimi'])


## Předzpracování dat

V této části si naše data předzpracujeme do podoby, ve které s nimi budeme pracovat. Filmy zde budeme chápat jako n-tice toho, co se nám bude pro doporučování hodit. 

V našem případě budeme hledat podobné filmy na základě žánrů. K tomu budeme potřebovat mít pro každý film základní popisné údaje (ID a název) a množinu žánrů, kterých daný film je; ideálně v číselné podobě, takže musíme tyto žánry (kategoriální hodnoty) převést na čísla.

Chápejme tedy nyní filmy coby řádky v tabulce, kde ve sloupečcích kromě popisných údajů najdeme sloupečky číselně vyjadřující, zda je film daného žánru či nikoliv. Jinými slovy, každý film bude mít přiřazen seznam hodnot vyjadřující pro daný sloupeček zda jde o film daného žánru (`1`) či nikoliv (`0`).


### Příklad

Filmy `Vykoupení z věznice Shawshang` a `Nedotknutelní` bychom mohli chápat následovně:

```js
[
    {
        '_id': '64d2a7963b7afeb6a7ff6b70',
        'name': 'Vykoupení z věznice Shawshank',
        'genres': ['Drama', 'Krimi', 'Vězení']
    },
    {
        '_id': '64d2a7f53b7afeb6a7ff6b77',
        'name': 'Nedotknutelní',
        'genres': ['Komedie', 'Drama', 'Životopisný']
    }
]
```

Po zpracování bychom očekávali něco podobného tomuto:

| ID | Název | Akční | Drama | Krimi | Komedie | Vězení | Životopisný |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 64d2a7963b7afeb6a7ff6b70 | Vykoupení z věznice Shawshank | 0 | 1 | 1 | 0 | 1 | 0 |
| 64d2a7f53b7afeb6a7ff6b77 | Nedotknutelní | 0 | 1 | 0 | 1 | 0 | 1 |



Tento princip převodu kategoriálních hodnot na numerické lze najít pod heslem _one-hot encoding_.

Pro tyto účely se nám bude hodit si předpočítat (resp. zjistit) všechny žánry, které se mezi našimi filmy objevují.

In [6]:
genres = []

for movie in movies:
    for genre in movie.genres:
        genres.append(genre)

genres = list(set(genres))

Zde provedeme samotné převedení filmů do námi požadované podoby.

In [7]:
def transform_movie_genre_to_number(movie: Movie, genre: str) -> int:
    """Převede žánr pro daný film na číslo. Pokud je daný film 
    daného žánru, vrací 1, jinak 0."""
    return 1 if movie.is_of_genre(genre) else 0


def movie_to_tuple(movie: Movie) -> tuple:
    """Převádí dodaný film na n-tici"""
    return tuple([
        movie.movie_id,
        movie.name,

        # Vrací se nám seznam čísel, my ho chceme rozprostřít, 
        # proto použijeme hvězdičkový operátor před získanou kolekcí
        *[transform_movie_genre_to_number(movie, genre) for genre in genres]
    ])

movie_tuples = [movie_to_tuple(movie) for movie in movies]

for movie_tuple in movie_tuples[:5]:
    print(movie_tuple)

('64d2a7963b7afeb6a7ff6b70', 'Vykoupení z věznice Shawshank', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
('64d2a7f53b7afeb6a7ff6b77', 'Nedotknutelní', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0)
('64d2b5d6aacb468a3ff1a9a8', 'Kmotr', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
('64d2bb9c22e576367f7b1f7a', 'Pelíšky', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
('64d528fa4ff07304e7d604e0', 'Forrest Gump', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)


### Mechanismy výpočtu podobnosti

Nyní se podívejme na mechanismy, pomocí kterých budeme počítat podobnost filmů. Jak je vidět z předchozího výpisu, filmy jsme si převedli do číselné podoby (tedy co se žánrů týče). Idea, se kterou budeme pracovat je, že tyto filmy pak jsou pro nás vlastně body v `n`-dimenzionálním prostoru, kde `n` odpovídá počtu všech žánrů napříč všemi filmy.

Na základě toho nám pak stačí počítat vzdálenosti mezi dvěma filmy (tedy body) a vyhledat ten nejpodobnější. K tomu použijeme následující dvě metriky:

- Euklidovská
- Kosínová

Euklidovská vzdálenost je základní metrikou, kterou můžeme použít pro počítání rozdílů mezi dvěma body v `n`-dimenzionálním prostoru. Využijeme následující vzorec, který lze snadno odvodit z pythagorovy věty pro použití v `n`-dimenzionálním prostoru:

$$d_{euklid} = \sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + \ldots + (x_n - y_n)^2}$$


Pomocí tohoto vzorce jsme schopni spočítat, že některé filmy jsou si podobné na základě žánrů.

In [8]:
def euclidean_distance(point1: tuple[float], point2: tuple[float]) -> float:
    """Spočítá euklidovskou vzdálenost mezi dvěma mnohadimenzionálními body.
    Pokud body nejsou stejné dimenzionality, vyhazuje chybu.
    """
    if len(point1) != len(point2):
        raise ValueError(f"Body musí být stejné dimenzionality: {len(point1)} != {len(point2)}")

    differences = []
    # Pro každou dimenzi
    for idx, value_in_dim in enumerate(point1):
        # Přidej absolutní rozdíl mezi hodnotami v dimenzi
        differences.append(abs(value_in_dim - point2[idx]))

    # Převeď absolutní rozdíly na své druhé mocniny
    squared_differences = [diff ** 2 for diff in differences]

    # Odmocni umocněné rozdíly
    return sum(squared_differences) ** 0.5

Kosínová vzdálenost je druhou významnou metrikou, která nám dokáže spočítátat podobnost dvou bodů v `n`-dimenzionálním prostoru. Pracuje s úhlem, který dva dané vektory vzájemně svírají.

Vzorec pro kosínovu větu říká, že je-li úhel svíraný mezi danými dvěma vektory nulový, návratová hodnota bude `1`. Samotný vzorec pak vypadá následovně:

$$cos(\phi) = \frac{\vec{u} \cdot \vec{v}}{|\vec{u}| \cdot |\vec{v}|}$$


My z tohoto pravidla vyjdeme a trochu si vzoreček pro potřeby výpočtu této vzdálenosti (_i.e._ podobnosti) upravíme:

$$d_{cosine} = 1 - \frac{\vec{u} \cdot \vec{v}}{|\vec{u}| \cdot |\vec{v}|}$$


Tím zajistíme, že čím menší velikost svíraného úhlu, tím více se výsledná hodnota blíží nule. Zároveň zde máme z povahy řešeného problému garantováno, že nedojde k extrémním situacím:
- pracujeme v I. kvadrantu (svíraný úhel nemůže být větší než 90°)
- neexistuje film bez přiřazeného žánru (neexistuje tedy nulový vektor a tedy nenastane situace, kdy bychom dělili nulou)

Dané body můžeme chápat jako vektory od počátku k danému bodu. Tím získáme výhodu, že jsou-li vektory lineárně závislé (ve zkratce že leží na stejné přímce, resp. $k \cdot \vec{u} = \vec{v}$), bude vzdálenost mezi nimi chápána jako 0 (tedy totožné). Samozřejmě zde můžeme zobecnit tezi na *"čím menší úhel dva libovolně velké vektory svírají, tím jsou si podobnější"*.

In [9]:
def cosine_distance(u: tuple[float], v: tuple[float]) -> float:
    """Spočítá kosínovou vzdálenost mezi dvěma mnohadimenzionálními vektory.
    Pokud vektory nejsou stejné dimenzionality, vyhazuje chybu. Stejně tak pokud
    je některý z vektorů tzv. nulový (o nulové délce) - dělili bychom nulou.
    """
    if len(u) != len(v):
        raise ValueError(f"Vektory musí být stejné dimenzionality: {len(u)} != {len(v)}")

    # Nejdříve si spočteme čitatele - součin dvou vektorů (tzv. dot product)
    dot_product = sum(u_n * v_n for u_n, v_n in zip(u, v))

    # Připravíme si pomocnou funkci pro výpočet délky vektoru (pomocí pythagorovy věty)
    vector_lenght = lambda vector: sum(x ** 2 for x in vector) ** 0.5

    u_len = vector_lenght(u)
    v_len = vector_lenght(v)

    if u_len == 0 or v_len == 0:
        raise ValueError(f"Byl dodán nulový vektor")

    return 1 - (dot_product / (u_len * v_len))

## Zpracování dat

### Párování dvojic

In [10]:
def couple_movies(movies_to_be_couples: list[tuple]) -> list[tuple[tuple, tuple]]:
    """Tato funkce vrátí všechny dvojice filmů."""
    couples = []

    for movie1 in movies_to_be_couples:
        for movie2 in movies_to_be_couples:
            # Přeskoč, pokud jde o ten samý film, jinak přidej dvojici
            if movie1 != movie2:
                couples.append((movie1, movie2))

    return couples

In [11]:
movie_couples = couple_movies(movie_tuples)

for movie_couple in movie_couples[:5]:
    print(movie_couple)

(('64d2a7963b7afeb6a7ff6b70', 'Vykoupení z věznice Shawshank', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), ('64d2a7f53b7afeb6a7ff6b77', 'Nedotknutelní', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0))
(('64d2a7963b7afeb6a7ff6b70', 'Vykoupení z věznice Shawshank', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), ('64d2b5d6aacb468a3ff1a9a8', 'Kmotr', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0))
(('64d2a7963b7afeb6a7ff6b70', 'Vykoupení z věznice Shawshank', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), ('64d2bb9c22e576367f7b1f7a', 'Pelíšky', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### Prostý výpočet euklidovské vzdálenosti nad filmy

V této části si zkusme vytvořit seznam doporučení, který bychom mohli použít. Využijme k tomu data, která jsme si připravili a euklidovskou vzdálenost mezi body.

In [12]:
def extract_point(movie_tuple: tuple) -> tuple:
    """Vyfiltruje pouze data reprezentující bod v n-dimenzionálním prostoru,
    tedy pouze souřadnice bodu získané z žánrů daného filmu."""
    return movie_tuple[2:]

In [13]:
distance_between_movies_plain_euclid = []

for movie_couple in movie_couples:
    m1_point = extract_point(movie_couple[0])
    m2_point = extract_point(movie_couple[1])
    
    distance_between_movies_plain_euclid.append(
        (
            # Názvy filmů
            movie_couple[0][1],
            movie_couple[1][1],

            # Euklidovská vzdálenost mezi filmy
            euclidean_distance(m1_point, m2_point)
        )
    )

In [14]:
# 20 nejlepších (nejpodobnějších) dvojic
sorted(distance_between_movies_plain_euclid, key=lambda movie: movie[2])[:20]

[('Vykoupení z věznice Shawshank', 'Kult hákového kříže', 0.0),
 ('Kmotr', 'Dvanáct rozhněvaných mužů', 0.0),
 ('Kmotr', 'Kmotr II', 0.0),
 ('Kmotr', 'Leon', 0.0),
 ('Kmotr', 'L. A. - Přísně tajné', 0.0),
 ('Kmotr', 'Butch Cassidy a Sundance Kid', 0.0),
 ('Kmotr', 'Město bohů', 0.0),
 ('Kmotr', 'Casino', 0.0),
 ('Kmotr', 'Motýlek', 0.0),
 ('Kmotr', 'Obvyklí podezřelí', 0.0),
 ('Kmotr', 'Mafiáni', 0.0),
 ('Kmotr', 'Tenkrát v Americe', 0.0),
 ('Kmotr', 'Obchodník se smrtí', 0.0),
 ('Forrest Gump', 'Podraz', 0.0),
 ('Forrest Gump', 'Obecná škola', 0.0),
 ('Forrest Gump', 'Zelená kniha', 0.0),
 ('Forrest Gump', 'Smrt krásných srnců', 0.0),
 ('Forrest Gump', 'Kolja', 0.0),
 ('Forrest Gump', 'Gympl', 0.0),
 ('Forrest Gump', 'Muž na Měsíci', 0.0)]

In [15]:
# 20 náhodně vybraných dvojic
random.sample(distance_between_movies_plain_euclid, 20)

[('Goldfinger', 'Vzhůru do oblak', 2.0),
 ('Obvyklí podezřelí', 'Sbal prachy a vypadni', 1.4142135623730951),
 ('Lara Croft - Tomb Raider', 'Medvěd', 2.23606797749979),
 ('Mission: Impossible', 'Avatar', 2.0),
 ('Medvěd', 'Žhavé výstřely', 2.449489742783178),
 ('Star Trek X: Nemesis', 'Skrytá čísla', 3.0),
 ('Transformers', 'Život je krásný', 2.8284271247461903),
 ('Na samotě u lesa', 'Leon', 2.0),
 ('Dnes neumírej', 'Czech Made Man', 2.8284271247461903),
 ('Město bohů', 'Císařův pekař - Pekařův císař', 2.23606797749979),
 ('25 let neviny', 'Kameňák', 1.7320508075688772),
 ('Jára Cimrman ležící, spící', 'Jak vycvičit draka', 2.0),
 ('Vesničko má středisková', 'Wolverine', 2.23606797749979),
 ('Rambo: První krev', 'Maska', 2.449489742783178),
 ('Podraz', 'Třináct životů', 1.4142135623730951),
 ('Rychle a zběsile', '300: Bitva u Thermopyl', 2.6457513110645907),
 ('Coco', 'Star Wars: Epizoda VI - Návrat Jediů', 2.23606797749979),
 ('Nedotknutelní', 'Dannyho parťáci', 1.7320508075688772),


Z těchto výpisů vidíme, že sice asi rozumíme tomu, proč jsou některé dvojice přiřazeny k sobě, ale ne vždy bychom tyto k sobě přiřadili. Důvodů může být více, například nedostatek dat. Jaká další data by se nám například mohla hodit? 
- Herci
- Země natočení
- Rok, v kterém byl film natočen
- Více kategorií o daném filmu

Všechny tyto atributy bychom mohli přepočítat pomocí výše uvedeného schématu a získali bychom přesnější doporučování.

Nicméně, zkusme na to jít ještě dalšími způsoby. Z povahy problému vidíme, že se pohybujeme pouze na vrcholech `n`-dimenzionální kostky o šířce strany `1` (film je nebo není daného žánru). Lze s tím něco udělat?

### Vážená euklidovská vzdálenost mezi filmy

Nyní uvažujme, že některé žánry mohou mít větší význam než jiné. Co kdybychom použili váhu významu? 

Jinými slovy, zamysleme se, jestli nese větší informaci fakt, že jsou oba filmy dramata (což je asi nejčastější žánr filmu), či skutečnost, že jsou oba filmy válečné (což je podstatně menší podmnožina snímků). 

Stejně tak se můžeme zamyslet nad otázkou, zda je-li film čistě komedie, jestli je to pro nás jeho důležitejší rys než když se bavíme o jiném filmu, který je zároveň válečnou muzikální komedií s dramatickými prvky.

#### Relevance žánru pro film

Zaměřme se nyní na druhý přístup, tedy kdy počítáme jak důležitý je daný žánr pro daný film. Zatím si vystačíme s tím, když pro každý žánr daného filmu v naší soustavě nul a jedniček přepočítáme pomocí vzorce:

$$\text{vážený indikátor žánru} = \frac{\text{indikátor žánru}}{\text{počet žánrů filmu}}$$

kde:
- `indikátor žánru` odpovídá právě dané hodnotě `1` nebo `0` (zda-li film je či není daného žánru)
- `počet žánrů filmu` odpovídá počtu přiřazených žánrů k danému filmu

In [16]:
weighted_movies_by_number_of_genres = []

for movie_tuple in movie_tuples:
    
    movie_point = extract_point(movie_tuple)

    # Zde jsou jen jedničky a nuly, můžeme je tedy snadno sečíst
    number_of_genres = sum(movie_point)

    # Přepočteme dané souřadnice
    movie_point = [(value / number_of_genres) for value in movie_point]

    # Uložíme do připraveného seznamu
    weighted_movies_by_number_of_genres.append(
        (
            movie_tuple[0],
            movie_tuple[1],
            *movie_point
        )
    )

for weighted in weighted_movies_by_number_of_genres[:3]:
    print(weighted)

('64d2a7963b7afeb6a7ff6b70', 'Vykoupení z věznice Shawshank', 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0)
('64d2a7f53b7afeb6a7ff6b77', 'Nedotknutelní', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0)
('64d2b5d6aacb468a3ff1a9a8', 'Kmotr', 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0)


In [17]:
weighted_movies_by_number_of_genres_euclid = []

for movie_couple in couple_movies(weighted_movies_by_number_of_genres):
    m1_point = extract_point(movie_couple[0])
    m2_point = extract_point(movie_couple[1])
    
    weighted_movies_by_number_of_genres_euclid.append(
        (
            # Názvy filmů
            movie_couple[0][1],
            movie_couple[1][1],

            # Euklidovská vzdálenost mezi filmy
            euclidean_distance(m1_point, m2_point)
        )
    )

In [18]:
# 20 nejlepších (nejpodobnějších) dvojic
sorted(weighted_movies_by_number_of_genres_euclid, key=lambda movie: movie[2])[:20]

[('Vykoupení z věznice Shawshank', 'Kult hákového kříže', 0.0),
 ('Kmotr', 'Dvanáct rozhněvaných mužů', 0.0),
 ('Kmotr', 'Kmotr II', 0.0),
 ('Kmotr', 'Leon', 0.0),
 ('Kmotr', 'L. A. - Přísně tajné', 0.0),
 ('Kmotr', 'Butch Cassidy a Sundance Kid', 0.0),
 ('Kmotr', 'Město bohů', 0.0),
 ('Kmotr', 'Casino', 0.0),
 ('Kmotr', 'Motýlek', 0.0),
 ('Kmotr', 'Obvyklí podezřelí', 0.0),
 ('Kmotr', 'Mafiáni', 0.0),
 ('Kmotr', 'Tenkrát v Americe', 0.0),
 ('Kmotr', 'Obchodník se smrtí', 0.0),
 ('Forrest Gump', 'Podraz', 0.0),
 ('Forrest Gump', 'Obecná škola', 0.0),
 ('Forrest Gump', 'Zelená kniha', 0.0),
 ('Forrest Gump', 'Smrt krásných srnců', 0.0),
 ('Forrest Gump', 'Kolja', 0.0),
 ('Forrest Gump', 'Gympl', 0.0),
 ('Forrest Gump', 'Muž na Měsíci', 0.0)]

In [19]:
# 20 náhodně vybraných dvojic
random.sample(weighted_movies_by_number_of_genres_euclid, 20)

[('Black Panther', 'Četa', 0.8660254037844386),
 ('Mamma Mia!', 'Million Dollar Baby', 0.9128709291752768),
 ('Poklad na Stříbrném jezeře', 'Vynález zkázy', 0.7071067811865476),
 ('Casino', 'Americký gangster', 0.5477225575051662),
 ('Auta', 'Temný rytíř povstal', 0.6708203932499369),
 ('Poslušně hlásím', 'Warrior', 1.0),
 ('Nespoutaný Django', 'Iluzionista', 0.8660254037844386),
 ('Počátek', 'Teorie všeho', 0.8660254037844386),
 ('Tanec s vlky', 'Pianista', 0.7071067811865476),
 ('Maska', '25 let neviny', 0.8366600265340756),
 ('Sbal prachy a vypadni', 'Monty Python a Svatý Grál', 1.0),
 ('Slunce, seno a pár facek', 'Na samotě u lesa', 0.7071067811865476),
 ('Kladivo na čarodějnice', 'L. A. - Přísně tajné', 0.7071067811865476),
 ('300: Bitva u Thermopyl', 'Pelíšky', 0.7637626158259733),
 ('Čtyři pokoje', 'Kult hákového kříže', 0.9128709291752769),
 ('Havel', 'John Wick', 0.7637626158259733),
 ('Úžasňákovi', 'Ace Ventura 2: Volání divočiny', 0.6172133998483678),
 ('Sám doma', 'Interste

Zde si můžeme všimnout, že dvojice filmů, které mají stejnou skladbu žánrů zůstávají stabilně ve svém určení (tedy `distance=0.0`). Změnily se však ty, které stejné žánry nemají. To není neočekávané chování - podle tohoto principu totiž je vlastně jedno, jaké váhy použiji (ve výchozím nastavení `weight=1`), protože vždy bude výsledek stejný. Liší se právě tam, když některému filmu nějaký ten žánr chybí.

Tomuto se říká tzv. _perfect match_. Pro nás to ale stále není zcela dokonalé, protože jsme naše doporučování nijak nezlepšili v kontextu množiny, která nás zajímá (tedy ty skutečně nejpodobnější filmy). Opět, bariérou pro nás v této ukázce je nedostatek dat.

### Prostá Kosínová podobnost filmů

Zkusme si nyní představit, že nepracujeme s euklidovskou vzdáleností (s body), nýbrž s vektory. Jak je uvedeno výše, bod můžeme vesele zaměnit za vektor mezi počátkem soustavy souřadnic a právě daným bodem. 

Pak můžeme chápat dva filmy jako vektory v nám známém `n`-rozměrném prostoru a můžeme počítat podobnost těchto vektorů přes vzájemně svíraný úhel. Čím menší úhel je mezi těmito vektory, tím podobnější tyto filmy jsou.

In [20]:
distance_between_movies_plain_cosine = []

for movie_couple in movie_couples:
    m1_point = extract_point(movie_couple[0])
    m2_point = extract_point(movie_couple[1])
    
    distance_between_movies_plain_cosine.append(
        (
            # Názvy filmů
            movie_couple[0][1],
            movie_couple[1][1],

            # Kosínová vzdálenost mezi filmy
            cosine_distance(m1_point, m2_point)
        )
    )

In [21]:
# 20 nejlepších (nejpodobnějších) dvojic
sorted(distance_between_movies_plain_cosine, key=lambda movie: movie[2])[:3]

[('Vykoupení z věznice Shawshank',
  'Kult hákového kříže',
  -2.220446049250313e-16),
 ('Pulp Fiction: Historky z podsvětí',
  'Zjizvená tvář',
  -2.220446049250313e-16),
 ('Dobyvatelé ztracené archy',
  'Indiana Jones a Poslední křížová výprava',
  -2.220446049250313e-16)]

Kvůli zaokrouhlovacím chybám dostáváme poněkud zvláštní výsledky. Asi se shodneme na tom, že bude snazší takto malá čísla považovat za hodnotu `0.0`

In [22]:
def normalize_to_zero(value: float, tolerance: float = 1e-15) -> float:
    """Převede příliš malá čísla na nulu (0.0)."""
    if abs(value) < tolerance:
        return 0.0
    return value

In [23]:
distance_between_movies_plain_cosine = [(c[0], c[1], normalize_to_zero(c[2])) for c in distance_between_movies_plain_cosine]

# 20 nejlepších (nejpodobnějších) dvojic
sorted(distance_between_movies_plain_cosine, key=lambda movie: movie[2])[:20]

[('Vykoupení z věznice Shawshank', 'Kult hákového kříže', 0.0),
 ('Kmotr', 'Dvanáct rozhněvaných mužů', 0.0),
 ('Kmotr', 'Kmotr II', 0.0),
 ('Kmotr', 'Leon', 0.0),
 ('Kmotr', 'L. A. - Přísně tajné', 0.0),
 ('Kmotr', 'Butch Cassidy a Sundance Kid', 0.0),
 ('Kmotr', 'Město bohů', 0.0),
 ('Kmotr', 'Casino', 0.0),
 ('Kmotr', 'Motýlek', 0.0),
 ('Kmotr', 'Obvyklí podezřelí', 0.0),
 ('Kmotr', 'Mafiáni', 0.0),
 ('Kmotr', 'Tenkrát v Americe', 0.0),
 ('Kmotr', 'Obchodník se smrtí', 0.0),
 ('Forrest Gump', 'Podraz', 0.0),
 ('Forrest Gump', 'Obecná škola', 0.0),
 ('Forrest Gump', 'Zelená kniha', 0.0),
 ('Forrest Gump', 'Smrt krásných srnců', 0.0),
 ('Forrest Gump', 'Kolja', 0.0),
 ('Forrest Gump', 'Gympl', 0.0),
 ('Forrest Gump', 'Muž na Měsíci', 0.0)]

In [24]:
# 20 náhodně vybraných dvojic
random.sample(distance_between_movies_plain_cosine, 20)

[('Pátý element', 'Jáchyme, hoď ho do stroje!', 0.6220355269907728),
 ('Predátor', 'Leon', 1.0),
 ('Dnes neumírej', 'Kmotr II', 0.683772233983162),
 ('Indiana Jones a Poslední křížová výprava', 'Zjizvená tvář', 1.0),
 ('Taxi', 'Anděl Páně', 1.0),
 ('Venom', 'Smrtonosná zbraň', 0.683772233983162),
 ('Čtyři pokoje', 'Mamma Mia!', 0.5917517095361371),
 ('Thor', 'Gran Torino', 1.0),
 ('E.T. - Mimozemšťan', 'Méďa', 1.0),
 ('Dannyho parťáci', 'Slunce, seno a pár facek', 0.29289321881345254),
 ('12 opic', 'Na samotě u lesa', 1.0),
 ('Sociální síť', 'Vlasy', 1.0),
 ('Slavnosti sněženek', 'Letec', 1.0),
 ('Vejška', 'Slavnosti sněženek', 0.0),
 ('Texaský masakr motorovou pilou', 'Poklad na Stříbrném jezeře', 1.0),
 ('Total Recall', 'Maska', 0.7418011102528389),
 ('Zelená Míle', 'T2 Trainspotting', 0.5917517095361371),
 ('Nelítostný souboj', 'Dokonalý trik', 1.0),
 ('Jáchyme, hoď ho do stroje!', 'Amélie z Montmartru', 0.42264973081037416),
 ('Trainspotting', 'Kokosy na sněhu', 0.7113248654051871)

V zásadě lze říci, že dvojice filmů, které nemají nic společného, jsou ohodnoceny číslem `1`. Čím více jsou si podobné (vektory svírají ostřejší úhel), tím blíže se jejich podobnost dostává hodnotě `0`. Opět, nevyřešili jsme problém s filmy, které mají stejné kategorie, ale dostáváme poněkud bližší vhled do celkové podobnosti.

Důvodem je například i fakt, že máme-li dva filmy `[Akční, Drama]` a `[Akční, Drama, Komedie]`, cítíme, že rozdíl nám tam dělá právě ta `Komedie` - jen a pouze ta. A z povahy věci také tušíme, že čím více těchto společných kategorií bude, tím méně bude výsledný úhel ovlivňen.

### Vážená kosínová vzdálenost nad filmy

Nyní zkusme pro úplnost analogicky to samé vážení, co jsme provedli pro euklidovskou vzdálenost.

In [25]:
weighted_movies_by_number_of_genres_cosine = []

for movie_couple in couple_movies(weighted_movies_by_number_of_genres):
    m1_point = extract_point(movie_couple[0])
    m2_point = extract_point(movie_couple[1])
    
    weighted_movies_by_number_of_genres_cosine.append(
        (
            # Názvy filmů
            movie_couple[0][1],
            movie_couple[1][1],

            # Euklidovská vzdálenost mezi filmy
            cosine_distance(m1_point, m2_point)
        )
    )

In [26]:
weighted_movies_by_number_of_genres_cosine = [(c[0], c[1], normalize_to_zero(c[2])) for c in weighted_movies_by_number_of_genres_cosine]

# 20 nejlepších (nejpodobnějších) dvojic
sorted(weighted_movies_by_number_of_genres_cosine, key=lambda movie: movie[2])[:20]

[('Vykoupení z věznice Shawshank', 'Kult hákového kříže', 0.0),
 ('Kmotr', 'Dvanáct rozhněvaných mužů', 0.0),
 ('Kmotr', 'Kmotr II', 0.0),
 ('Kmotr', 'Leon', 0.0),
 ('Kmotr', 'L. A. - Přísně tajné', 0.0),
 ('Kmotr', 'Butch Cassidy a Sundance Kid', 0.0),
 ('Kmotr', 'Město bohů', 0.0),
 ('Kmotr', 'Casino', 0.0),
 ('Kmotr', 'Motýlek', 0.0),
 ('Kmotr', 'Obvyklí podezřelí', 0.0),
 ('Kmotr', 'Mafiáni', 0.0),
 ('Kmotr', 'Tenkrát v Americe', 0.0),
 ('Kmotr', 'Obchodník se smrtí', 0.0),
 ('Forrest Gump', 'Podraz', 0.0),
 ('Forrest Gump', 'Obecná škola', 0.0),
 ('Forrest Gump', 'Zelená kniha', 0.0),
 ('Forrest Gump', 'Smrt krásných srnců', 0.0),
 ('Forrest Gump', 'Kolja', 0.0),
 ('Forrest Gump', 'Gympl', 0.0),
 ('Forrest Gump', 'Muž na Měsíci', 0.0)]

In [27]:
# 20 náhodně vybraných dvojic
random.sample(weighted_movies_by_number_of_genres_cosine, 20)

[('Policejní akademie', 'Čas probuzení', 1.0),
 ('The Big Lebowski', 'Pianista', 1.0),
 ('Samotáři', 'L. A. - Přísně tajné', 0.591751709536137),
 ('Zachraňte vojína Ryana', 'Transformers', 1.0),
 ('Přednosta stanice', 'Whiplash', 1.0),
 ('U pokladny stál...', 'Vrchní, prchni', 0.0),
 ('Temný rytíř', 'Úžasňákovi', 0.42264973081037427),
 ('Star Wars: Epizoda VI - Návrat Jediů', 'Mariňák', 1.0),
 ('Barbie', 'Piráti z Karibiku: Truhla mrtvého muže', 0.42264973081037427),
 ('Aquaman', 'Muži v černém', 0.7763932022500211),
 ('Sherlock Holmes: Hra stínů', 'Cesta do pravěku', 0.7642977396044841),
 ('Rozpuštěný a vypuštěný', 'Sociální síť', 1.0),
 ('Kung Fu Panda', 'Četník ze Saint Tropez', 0.7113248654051871),
 ('Whiplash', 'Hodný, zlý a ošklivý', 1.0),
 ('Team America: Světovej policajt', 'Obchodník se smrtí', 1.0),
 ('Iron Man', 'Kill Bill', 0.6666666666666667),
 ('Kokosy na sněhu', 'Rain Man', 1.0),
 ('Titanic', 'Apollo 13', 0.6666666666666667),
 ('Pán prstenů: Návrat krále', 'VALL-I', 0.64

## Závěr

Cílem bylo zjistit, které filmy spolu souvisí pomocí přístupu předpočítání znalostí o podobných filmech. 

V praxi by se takto vydolovaných znalostí dalo použít omezením hodnoty podobnosti (například filmy, které jsou vzdálenější než `0.35` nedoporučovat) a nasazením do systému. Jakmile by uživatel projevil zájem o některý film, nám pak stačí vyhledat námi zjištěné podobné snímky a ty dodat jako doporučit.

Tento přístup je (jak sami vidíte) velmi jednoduchý a nepříliš výkonově náročný. Nicméně má (alespoň v této ukázce) své výzvy a je silně závislý na tom, jak důsledně pracujeme s atributy položek. 

Tedy pro zopakování - lepších doporučovacích výsledků bychom dosáhli s větším objemem dat (více filmů, více atributů).

## Pro zájemce

Zkuste si implementovat vážení filmů pomocí relativní významnosti žánru, tedy že drama (coby jeden z nejčetnějších žánrů) nám dává menší význam než třeba filmy psychologické či válečné.